# Imports

In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

2022-10-25 18:43:30.062452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 18:43:39.409395: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 18:43:58.200230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pdda_lab_user/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-10-25 18:43:58.200384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugi

In [7]:
path = "data/UTKFace"
images = []
age = []
gender = []
for img in os.listdir(path):
  ages = img.split("_")[0] #The first value before the _ is age
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(np.array(img))
  age.append(np.array(ages))

In [8]:
age = np.array(age,dtype=np.int64)
images = np.array(images)   

In [9]:
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(images, age, random_state=42)

# Model

In [10]:
age_model = Sequential()
age_model.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(200,200,3)))
age_model.add(MaxPool2D(pool_size=3, strides=2))

age_model.add(Conv2D(128, kernel_size=3, activation='relu'))
age_model.add(MaxPool2D(pool_size=3, strides=2))
              
age_model.add(Conv2D(256, kernel_size=3, activation='relu'))
age_model.add(MaxPool2D(pool_size=3, strides=2))

age_model.add(Conv2D(512, kernel_size=3, activation='relu'))
age_model.add(MaxPool2D(pool_size=3, strides=2))

age_model.add(Flatten())
age_model.add(Dropout(0.2))
age_model.add(Dense(512, activation='relu'))

# Use 1 node and linear activation as this isn't a classification problem
age_model.add(Dense(1, activation='linear', name='age'))
              
age_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print(age_model.summary())              
                           
history_age = age_model.fit(x_train_age, y_train_age,
                        validation_data=(x_test_age, y_test_age), epochs=50)

age_model.save('age_model_50epochs.h5')

2022-10-25 19:02:08.050230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 19:02:32.019023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10787 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 98, 98, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 47, 47, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 256)      0

2022-10-25 19:03:10.176406: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-10-25 19:03:58.357174: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


556/556 [==============================] - 264s 302ms/step - loss: 62642.6484 - mae: 25.6811 - val_loss: 254.8404 - val_mae: 12.8334
Epoch 2/50
556/556 [==============================] - 164s 294ms/step - loss: 235.6303 - mae: 11.7324 - val_loss: 213.6995 - val_mae: 10.7306
Epoch 3/50
556/556 [==============================] - 164s 294ms/step - loss: 205.2813 - mae: 10.9601 - val_loss: 171.3535 - val_mae: 9.9513
Epoch 4/50
556/556 [==============================] - 164s 295ms/step - loss: 174.5600 - mae: 10.0223 - val_loss: 147.5504 - val_mae: 8.9191
Epoch 5/50
556/556 [==============================] - 164s 295ms/step - loss: 149.0925 - mae: 9.2242 - val_loss: 155.8577 - val_mae: 9.1276
Epoch 6/50
556/556 [==============================] - 164s 294ms/step - loss: 140.6497 - mae: 8.9129 - val_loss: 122.9453 - val_mae: 8.2118
Epoch 7/50
556/556 [==============================] - 164s 294ms/step - loss: 130.2682 - mae: 8.5735 - val_loss: 151.5500 - val_mae: 9.8527
Epoch 8/50
556/556 [===

In [17]:
from keras.models import load_model
#Test the model
my_model = load_model('age_model_50epochs.h5', compile=False)


predictions = my_model.predict(x_test_age)
y_pred = (predictions>= 0.5).astype(int)[:,0]

from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test_age, y_pred))


186/186 [==============================] - 15s 82ms/step
Accuracy =  0.02986333727011979


In [14]:
from time import sleep
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image

In [15]:
face_classifier=cv2.CascadeClassifier('haarcascades_models/haarcascade_frontalface_default.xml')

[ERROR:0@10827.227] global /io/opencv/modules/core/src/persistence.cpp (505) open Can't open file: 'haarcascades_models/haarcascade_frontalface_default.xml' in read mode


In [16]:
age_model = load_model('age_model_50epochs.h5')

In [ ]:
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    labels=[]
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray=gray[y:y+h,x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        #Get image ready for prediction
        roi=roi_gray.astype('float')/255.0  #Scale
        roi=img_to_array(roi)
        roi=np.expand_dims(roi,axis=0)  #Expand dims to get it ready for prediction (1, 48, 48, 1)
        
        roi_color=frame[y:y+h,x:x+w]
        roi_color=cv2.resize(roi_color,(200,200),interpolation=cv2.INTER_AREA)

        age_predict = age_model.predict(np.array(roi_color).reshape(-1,200,200,3))
        age = round(age_predict[0,0])
        age_label_position=(x+h,y+h)
        cv2.putText(frame,"Age="+str(age),age_label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        
   
    cv2.imshow('Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()